In [4]:
import mplpub
import numpy as np
from ase import Atoms
from ase.io import read
from ase.spacegroup.symmetrize import FixSymmetry
from ase.units import GPa, kB
from calorine.calculators import CPUNEP
from calorine.tools import get_force_constants, relax_structure
from matplotlib import pyplot as plt, colormaps
from pandas import DataFrame, read_json, concat as pd_concat
from phonopy import load, Phonopy, PhonopyQHA
from phonopy.structure.atoms import PhonopyAtoms
from phonopy.units import Avogadro, EvTokJmol, kb_J, THzToEv
from seekpath import get_explicit_k_path
from tqdm import tqdm

R = kb_J * Avogadro
mplpub.setup(template='acs')

In [5]:
def get_props(
    ph, classical,
    mesh=[40, 40, 40],
    t_min=1, t_max=600, t_step=1,
    freq_min=-5, freq_max=20, freq_pitch=0.05,
):
    # evaluate frequencies on the specified q-point mesh
    ph.run_mesh(mesh)

    # compute thermal properties
    ph.run_thermal_properties(t_min=t_min, t_max=t_max, t_step=t_step, classical=classical)

    # turn results into DataFrame
    tp_dict = ph.get_thermal_properties_dict()
    df_thermo = DataFrame(tp_dict)
    df_thermo['temperature'] = df_thermo.temperatures
    del df_thermo['temperatures']

    # add columns with properties in more convenient units for plotting
    natoms = len(get_structure(ph))
    df_thermo['free_energy_eV'] = df_thermo.free_energy / EvTokJmol / natoms
    df_thermo['entropy_kB'] = df_thermo.entropy / R / natoms
    df_thermo['heat_capacity_kB'] = df_thermo.heat_capacity / R / natoms

    # density of states
    ph.run_total_dos(freq_min=freq_min, freq_max=freq_max, freq_pitch=freq_pitch)
    dos_dict = ph.get_total_dos_dict()
    df_dos = DataFrame(dos_dict)
    df_dos['frequency'] = df_dos.frequency_points
    del df_dos['frequency_points']
    df_dos.total_dos /= natoms

    return df_thermo, df_dos

In [6]:
def get_structure(phonon):
    structure_ph = phonon.primitive
    structure_ase = Atoms(symbols=structure_ph.symbols, cell=structure_ph.cell,
                          scaled_positions=structure_ph.scaled_positions, pbc=True)
    return structure_ase

In [7]:
def labels_to_latex(labels_in):
    labels = [r'$\UG{\Gamma}$' if m == 'GAMMA' else m for m in labels_in]
    labels = [m.replace('SIGMA', r'$\UG{\Sigma}$') if m.startswith('SIGMA') else m for m in labels]
    labels = [m.replace('_', '$_') + '$' if '_' in m else m for m in labels]
    return labels

colors = {
    'BaZrS3-spg140-I4mcm': 'C0',
    'BaZrS3-spg62-Pnma': 'C1',
    'BaZrS3-spg62-Pnma-DELTA': 'C2',
}

labels = {
    'BaZrS3-spg140-I4mcm': 'I4/mcm',
    'BaZrS3-spg62-Pnma': 'Pnma',
    'BaZrS3-spg62-Pnma-DELTA': 'delta',
}

In [8]:
xc = 'HSE'
model_filename = 'nep.txt'
supercell_matrix = [4, 4, 4]

phonons = {}
for sname in labels:
    print(f'sname: {sname}')
    phonons[sname] = {}
    for volsc in tqdm(np.arange(0.92, 1.11, 0.01)):
        volsc = np.around(volsc, decimals=3)
        outfile = f'runs-{xc}/phonon-{sname}-volsc{volsc:.3f}.yaml'
        try:
            ph = load(outfile)
        except:
            structure = read(f'structures/{sname}.xyz')
            cell = structure.cell[:] * volsc ** (1 / 3)
            structure.set_cell(cell, scale_atoms=True)
            calculator = CPUNEP(model_filename)
            structure.calc = calculator
            structure.set_constraint(FixSymmetry(structure))
            relax_structure(structure, constant_volume=True)
            ph = get_force_constants(structure, calculator, supercell_matrix=supercell_matrix)
            ph.save(outfile)

        phonons[sname][volsc] = ph

sname: BaZrS3-spg140-I4mcm


100%|██████████| 20/20 [00:12<00:00,  1.61it/s]


sname: BaZrS3-spg62-Pnma


100%|██████████| 20/20 [00:24<00:00,  1.22s/it]


sname: BaZrS3-spg62-Pnma-DELTA


  0%|          | 0/20 [00:00<?, ?it/s]/Users/w21013885/anaconda3/envs/nep/lib/python3.11/site-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.20905707068110413
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'
/Users/w21013885/anaconda3/envs/nep/lib/python3.11/site-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.21807817343657354
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'
/Users/w21013885/anaconda3/envs/nep/lib/python3.11/site-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjust_cell may be ill behaved with large deformation gradient step 0.2214027581541469
  warnings.warn('FixSymmetry adjust_cell may be ill behaved with'
/Users/w21013885/anaconda3/envs/nep/lib/python3.11/site-packages/ase/spacegroup/symmetrize.py:217: UserWarning: FixSymmetry adjus

In [9]:
volsc = 1.0

paths = {}
dfs_phonons = {}
for sname in phonons:
    phonon = phonons[sname][volsc]

    structure = get_structure(phonon)
    structure_tuple = (structure.cell[:], structure.get_scaled_positions(), structure.numbers)
    path = get_explicit_k_path(structure_tuple)
    paths[sname] = path.copy()

    phonon.run_band_structure([path['explicit_kpoints_rel']])
    band = phonon.get_band_structure_dict()

    df = DataFrame(band['frequencies'][0])
    df.index = path['explicit_kpoints_linearcoord']
    dfs_phonons[sname] = df 

In [10]:
ncols = len(dfs_phonons)

fig, axes = plt.subplots(
    figsize=(4.2, 2.1),
    ncols=len(dfs_phonons),
    sharey=True,
    dpi=200,
)

for icol, (sname, df) in enumerate(dfs_phonons.items()):
    ax = axes[icol]
    path = paths[sname]

    ax.set_title(labels[sname])
    for col in df.columns:
        ax.plot(df.index, df[col], color=colors[sname], alpha=0.3)
    ax.set_xlim(df.index.min(), df.index.max())

    if icol == 0:
        ax.set_ylabel('Frequency (THz)')
    if icol == ncols - 1:
        ax2 = ax.twinx()
        ax2.set_ylabel('Frequency (meV)')
        ax2.set_ylim(THzToEv * 1e3 * np.array(ax.get_ylim()))

    qpts_labels = labels_to_latex(path['explicit_kpoints_labels'])
    df_path = DataFrame(dict(labels=qpts_labels,
                             positions=path['explicit_kpoints_linearcoord']))
    df_path.drop(df_path.index[df_path.labels == ''], axis=0, inplace=True)
    ax.set_xticks(df_path.positions)
    ax.set_xticklabels(df_path.labels)
    for xp in df_path.positions:
        ax.axvline(xp, color='0.8')

fig.tight_layout()
fig.subplots_adjust(hspace=0, wspace=0.06) 

RuntimeError: Failed to process string with tex because latex could not be found

Error in callback <function _draw_all_if_interactive at 0x148aa8c20> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 840x420 with 4 Axes>